# Assignment Day 9

## Team members:
- Samuel Nebgen s6sanebg@uni-bonn.de
- Muhammad Humza Arain s27marai@uni-bonn.de
- Julian Meigen s82jmeig@uni-bonn.de

## 18.09.2025

Contributions were made by all team members in around the same amount, either based on discussions or coding.

# Task 1: Load the Mutagenicity dataset from TUDatasets (https://chrsmrrs.github.io/datasets/docs/datasets/)


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
from torch_geometric.datasets import TUDataset

In [ ]:
save_data_path = os.path.join("..", "data", "interim", "TUDataset", "MUTAG")
dataset = TUDataset(root=save_data_path, name="MUTAG")

Processing...
Done!


In [ ]:
dataset

MUTAG(188)

# Task 2: Visualize the data by drawing the molecule graph of one sample.


# Task 3: Train a GNN model with 5 convolutional layers with pytorch-geometric.


## a. Define the model.


## b. Define the train and test function.


## c. Train the model for 100 epochs.


# Task 4: Explain the predictions using the captum package (https://captum.ai/) with the method Integrated Gradients.


## a. Familiarize yourself with the method, and the captum package (https://captum.ai/docs/extension/integrated_gradients). Summarize the method in your own words (5-8sentences).



## b. Calculate the attributions of the input to the model’s predictions.


## c. Visualize your results. Which were the most important inputs for a prediction?


# Task 5: Use the GNNExplainer from pytorch-geometric to explain the model’s predictions.


## a. Familiarize yourself with the method (https://arxiv.org/pdf/1903.03894 ). Summarize the method in your own words (5-8 sentences).


## b. Visualize your results. Choose an interesting prediction and show the most important input.